# Bac à sable RAG

Ce notebook part du principe que la _vector database_ est déjà prête, c'est-à-dire que les étapes suivantes ont déjà été faites:

<div>
<img src="https://python.langchain.com/assets/images/rag_indexing-8160f90a90a33253d0154659cf7d453f.png" width="500"/>
</div>

Nous nous intéressons à celles-ci:

<div>
<img src="https://python.langchain.com/assets/images/rag_retrieval_generation-1046a4668d6bb08786ef73c56d4f228a.png" width="500"/>
</div>


In [2]:
import logging
import os
import s3fs

from langchain.schema.runnable.config import RunnableConfig
from langchain_core.prompts import PromptTemplate

from src.chain_building.build_chain import build_chain
from src.chain_building.build_chain_validator import build_chain_validator
from src.config import CHATBOT_TEMPLATE, EMB_MODEL_NAME
from src.db_building import (
    load_retriever,
    load_vector_database
)
from src.model_building import build_llm_model
from src.utils.formatting_utilities import add_sources_to_messages, str_to_bool

# Logging configuration
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s %(message)s",
    datefmt="%Y-%m-%d %I:%M:%S %p",
    level=logging.DEBUG,
)

# Remote file configuration
os.environ['MLFLOW_TRACKING_URI'] = "https://projet-llm-insee-open-data-mlflow.user.lab.sspcloud.fr/"
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": f"""https://{os.environ["AWS_S3_ENDPOINT"]}"""})

# PARAMETERS --------------------------------------

os.environ['UVICORN_TIMEOUT_KEEP_ALIVE'] = "0"

model = os.getenv("LLM_MODEL_NAME")
CHROMA_DB_LOCAL_DIRECTORY = "./data/chroma_db"
CLI_MESSAGE_SEPARATOR = f"{80*'-'} \n"
quantization = True
DEFAULT_MAX_NEW_TOKENS = 10
DEFAULT_MODEL_TEMPERATURE = 1
embedding = os.getenv("EMB_MODEL_NAME", EMB_MODEL_NAME)

model_id = "meta-llama/Llama-3.2-3B-Instruct"
LLM_MODEL = os.getenv("LLM_MODEL_NAME", "meta-llama/Llama-3.2-3B-Instruct")
LLM_MODEL = "microsoft/Phi-3.5-mini-instruct"
QUANTIZATION = os.getenv("QUANTIZATION", True)
MAX_NEW_TOKENS = int(os.getenv("MAX_NEW_TOKENS", DEFAULT_MAX_NEW_TOKENS))
MODEL_TEMPERATURE = int(os.getenv("MODEL_TEMPERATURE", DEFAULT_MODEL_TEMPERATURE))
RETURN_FULL_TEXT = os.getenv("RETURN_FULL_TEXT", True)
DO_SAMPLE = os.getenv("DO_SAMPLE", True)
DATABASE_RUN_ID = "32d4150a14fa40d49b9512e1f3ff9e8c"


/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Import de la database et du modèle génératif

### Base de données vectorielle

In [3]:
import os
from src.model_building.fetch_llm_model import cache_model_from_hf_hub
from utils import retrieve_db_from_cache

EMB_MODEL_NAME = "OrdalieTech/Solon-embeddings-large-0.1"
LLM_MODEL = "mistralai/Mistral-7B-Instruct-v0.3"

hf_token = os.environ["HF_TOKEN"]
s3_token = os.environ["AWS_SESSION_TOKEN"]

cache_model_from_hf_hub(EMB_MODEL_NAME, hf_token=hf_token, s3_token=s3_token)
cache_model_from_hf_hub(LLM_MODEL, hf_token=hf_token, s3_token=s3_token)

db = retrieve_db_from_cache(filesystem=fs, run_id=DATABASE_RUN_ID)

2024-11-20 15:36:35,603 - INFO - Destination path /tmp/mlflow/32d4150a14fa40d49b9512e1f3ff9e8c/chroma exists. Skipping download because force is set to False.


Model OrdalieTech/Solon-embeddings-large-0.1 found in local cache. 
Model mistralai/Mistral-7B-Instruct-v0.3 found in local cache. 


2024-11-20 15:36:35,977 - INFO - Load pretrained SentenceTransformer: OrdalieTech/Solon-embeddings-large-0.1
/home/onyxia/work/llm-open-data-insee/src/db_building/build_database.py:112: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(
2024-11-20 15:36:38,552 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-11-20 15:36:38,729 - WARNING - ⚠️ It looks like you upgraded from a version below 0.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.
2024-11-20 15:36:38,736 - INFO - The database (collection insee_data) has been reloaded from directory /tmp/mlflow/32d415

In [4]:
f"Nombre de documents dans la vector db: {len(db.get()['documents'])}"

'Nombre de documents dans la vector db: 287086'

### Modèle génératif

In [5]:
from langchain_community.llms import VLLM

MAX_NEW_TOKEN = 8192
TEMPERATURE = 0.2
REP_PENALTY = 1.1
TOP_P = 0.8

hf_token = os.environ["HF_TOKEN"]
s3_token = os.environ["AWS_SESSION_TOKEN"]

cache_model_from_hf_hub(EMB_MODEL_NAME, hf_token=hf_token, s3_token=s3_token)
cache_model_from_hf_hub(LLM_MODEL, hf_token=hf_token, s3_token=s3_token)

llm = VLLM(
        model=LLM_MODEL,
        max_new_tokens=MAX_NEW_TOKEN,
        top_p=TOP_P,
        temperature=TEMPERATURE,
        rep_penalty=REP_PENALTY,
    )

Model OrdalieTech/Solon-embeddings-large-0.1 found in local cache. 
Model mistralai/Mistral-7B-Instruct-v0.3 found in local cache. 
WARNING 11-20 15:36:50 cuda.py:23] You are using a deprecated `pynvml` package. Please install `nvidia-ml-py` instead, and make sure to uninstall `pynvml`. When both of them are installed, `pynvml` will take precedence and cause errors. See https://pypi.org/project/pynvml for more information.


2024-11-20 15:36:51,392	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


GatedRepoError: 401 Client Error. (Request ID: Root=1-673e0213-687cfbca344f8d1c56400f61;52afff2c-db70-46d7-a620-c95db14b4c9f)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3/resolve/main/sentence_bert_config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.3 is restricted. You must have access to it and be authenticated to access it. Please log in.

## Transform vector database into retriever

In [ ]:
retriever, vectorstore = load_retriever(
                emb_model_name=embedding,
                persist_directory=CHROMA_DB_LOCAL_DIRECTORY,
                vectorstore=db,
                retriever_params={
                    "search_type": "similarity",
                    "search_kwargs": {"k": 30}
                },
            )

2024-11-20 02:21:02 PM vectorstore being provided, skipping the reloading


In [ ]:
retriever.invoke("Chiffres du chômage")[:5]

Batches: 100%|██████████| 1/1 [00:00<00:00, 103.74it/s]


[Document(metadata={'Header 1': 'Chômage et halo autour du chômage en 2019', 'Header 2': 'Résumé :', 'Header 3': "Téléchargement des tableaux à l'unité", 'Header 4': 'Chômage', 'categorie': 'Chiffres détaillés', 'collection': '', 'dateDiffusion': '2020-06-23 12:00', 'libelleAffichageGeo': 'France', 'theme': 'Emploi – Population active', 'titre': 'Chômage et halo autour du chômage en 2019', 'url': 'https://www.insee.fr/fr/statistiques/4498582'}, page_content='#### Chômage'),
 Document(metadata={'Header 1': 'Emploi-Chômage', 'categorie': 'Publications pour expert', 'collection': 'Note de conjoncture', 'dateDiffusion': '2020-12-15 17:00', 'libelleAffichageGeo': 'France', 'theme': 'Économie générale (inflation, PIB, dette,...)', 'titre': 'Emploi-Chômage', 'url': 'https://www.insee.fr/fr/statistiques/4653872'}, page_content='# Emploi-Chômage'),
 Document(metadata={'Header 1': 'Chômage', 'categorie': 'Publications pour expert', 'collection': 'Note de conjoncture', 'dateDiffusion': '2017-03-1

In [6]:
retriever.batch(["Quelle est la définition du PIB ?", "Où sont les chiffres de l'inflation ?"])

NameError: name 'retriever' is not defined

## RAG mode: on 

In [34]:
from utils import (
    format_docs,
    create_prompt_from_instructions,
    retrieve_db_from_cache,
)

system_instructions = """
Tu es un assistant spécialisé dans la statistique publique. Tu réponds à des questions concernant les données de l'Insee, l'institut national statistique Français.

Réponds en FRANCAIS UNIQUEMENT. Utilise une mise en forme au format markdown.

En utilisant UNIQUEMENT les informations présentes dans le contexte, réponds de manière argumentée à la question posée.

La réponse doit être développée et citer ses sources (titre et url de la publication) qui sont référencées à la fin. Cite notamment l'url d'origine de la publication, dans un format markdown.

Cite 5 sources maximum.

Tu n'es pas obligé d'utiliser les sources les moins pertinentes. 

Si tu ne peux pas induire ta réponse du contexte, ne réponds pas.

Voici le contexte sur lequel tu dois baser ta réponse :
Contexte: {context}
"""

question_instructions = """
Voici la question à laquelle tu dois répondre :
Question: {question}

Réponse:
"""


prompt = create_prompt_from_instructions(system_instructions, question_instructions)


In [37]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

## Imbrication dans langchain
rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

In [38]:
rag_chain.invoke("Chiffres du chômage")

Processed prompts: 100%|██████████| 1/1 [00:13<00:00, 13.11s/it, est. speed input: 675.99 toks/s, output: 78.27 toks/s]


"1. Le chômage en France est mesuré par le taux de chômage, qui est le rapport entre le nombre de chômeurs au sens du BIT et le nombre de personnes en emploi ou au chômage. Le BIT (Bénéficiaire d'allocations chômage) est une personne qui reçoit des allocations chômage, c'est-à-dire des allocations de l'État pour couvrir ses frais de subsistance pendant qu'elle est au chômage.\n\n    2. Le taux de chômage en France a été stable depuis plusieurs années, mais il varie selon les régions et les catégories socioprofessionnelles. Par exemple, en 2019, le taux de chômage en France métropolitaine était de 7,7%, mais il était plus élevé en région Île-de-France (9,3%) et plus bas en région Centre-Val de Loire (6,1%).\n\n    3. Les chômeurs en France sont majoritairement des hommes (54,6% en 2019), mais leur proportion est en baisse depuis plusieurs années. Les chômeurs sont également majoritairement des personnes de 30 à 49 ans (68,1% en 2019), mais leur proportion est en hausse depuis plusieurs 

In [40]:
for chunk in rag_chain.stream("A quelle échelle puis-je trouver le taux de chîmage ? "):
    print(chunk, end="", flush=True)

Processed prompts: 100%|██████████| 1/1 [00:07<00:00,  7.83s/it, est. speed input: 1743.61 toks/s, output: 66.77 toks/s]

 Le taux de chômage peut être trouvé à l'échelle de la région, du département ou de la commune. Il est calculé en divisant le nombre de personnes chômeures par le nombre de personnes actives de la population.

     Pour trouver le taux de chômage à l'échelle de la région, il est nécessaire de consulter les données de l'Insee sur le taux de chômage régional. Par exemple, pour la région Grand Est, vous pouvez consulter la publication "Taux de chômage régional" sur le site de l'Insee : https://www.insee.fr/fr/statistiques/7931214

     Pour trouver le taux de chômage à l'échelle du département, il est nécessaire de consulter les données de l'Insee sur le taux de chômage départemental. Par exemple, pour le département de la Nièvre, vous pouvez consulter la publication "Taux de chômage départemental" sur le site de l'Insee : https://www.insee.fr/fr/statistiques/5370760

     Pour trouver le taux de chômage à l'échelle de la commune, il est nécessaire de consulter les données de l'Insee sur 